## Demo

In [1]:
from main import LateEmbedding

text = [
    "谢良兴国县长冈乡塘石村人。生于一九一五年四月。一九三○年六月参加中国工农红军，同年七月加人中国共产主义青年团",
    "解放战争时期，任陕甘宁联防军政治部组织部副部长，联防军后勤部政治部主任，冀鲁豫军区政治部组织部部长",
    "他因战伤截肢，人称“独脚将军。”获一级八一勋章、二级独立自由勋章、一级解放勋章和一级红星功勋荣誉章。"
]

embeddings = LateEmbedding().main(text)

Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
for i, (e, t) in enumerate(zip(embeddings, text)):
    print("文本：", t)
    print("Embedding：", e)

文本： 谢良兴国县长冈乡塘石村人。生于一九一五年四月。一九三○年六月参加中国工农红军，同年七月加人中国共产主义青年团
Embedding： [-1.05553043e+00  1.07637382e+00 -6.32273018e-01  6.09288990e-01
  9.30566862e-02 -1.20668781e+00  1.69311479e-01  4.92189050e-01
  1.28904617e+00 -9.05766010e-01 -5.27602732e-01  1.21311165e-01
 -6.76061273e-01  5.52891850e-01 -7.69335866e-01  6.61602497e-01
  1.16031682e+00  1.01493621e+00  1.06517553e+00  7.27962732e-01
  1.00545847e+00  6.26588225e-01 -3.23575169e-01  2.84039438e-01
 -4.66303639e-02  4.21039313e-01  4.19754446e-01 -1.05529773e+00
 -4.87552732e-01  7.45626152e-01 -4.59631205e-01 -8.13475028e-02
  1.27096027e-01  3.21923018e-01  1.26388216e+00 -2.85874695e-01
  5.49125075e-01  5.96240819e-01 -3.04597318e-01  6.42636716e-02
 -6.48993552e-01 -6.94985867e-01  6.40225351e-01  8.08206499e-01
 -5.54198503e-01  1.29071578e-01 -8.22851121e-01  3.27642322e-01
 -7.18944848e-01  9.05982196e-01  1.21521384e-01  4.05951262e-01
  5.43641865e-01  6.35557473e-01  6.94356561e-01  4.62060660e-01
 -1.79328823e+00  2.

## 比较传统chunking和late chunking效果

In [3]:
text = [
    "谢良兴国县长冈乡塘石村人。生于一九一五年四月。一九三○年六月参加中国工农红军，同年七月加人中国共产主义青年团",
    "解放战争时期，任陕甘宁联防军政治部组织部副部长，联防军后勤部政治部主任，冀鲁豫军区政治部组织部部长",
    "他因战伤截肢，人称“独脚将军。”获一级八一勋章、二级独立自由勋章、一级解放勋章和一级红星功勋荣誉章。"
]

In [4]:
# 传统chunking方法
from sentence_transformers import SentenceTransformer

trditional_model = SentenceTransformer("Alibaba-NLP/gte-multilingual-base", trust_remote_code=True)
embeddings_traditional_chunking = trditional_model.encode(text)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
# late chunking方法
embeddings = LateEmbedding().main(text)

Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
# 评估cosine similarity
import numpy as np

cos_sim = lambda x, y: np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))
query = '谁被称为称独脚将军'
query_embedding = trditional_model.encode(query)

print(f'query: {query}')
print("===")
for chunk, late_embedding, trad_embedding in zip(text, embeddings, embeddings_traditional_chunking):
    print(cos_sim(query_embedding, trad_embedding), f'trational: ("{chunk}"):')
    print(cos_sim(query_embedding, late_embedding), f'late: ("{chunk}"):')
    print("===")

query: 谁被称为称独脚将军
===
0.5242378 trational: ("谢良兴国县长冈乡塘石村人。生于一九一五年四月。一九三○年六月参加中国工农红军，同年七月加人中国共产主义青年团"):
0.5375414 late: ("谢良兴国县长冈乡塘石村人。生于一九一五年四月。一九三○年六月参加中国工农红军，同年七月加人中国共产主义青年团"):
===
0.5356585 trational: ("解放战争时期，任陕甘宁联防军政治部组织部副部长，联防军后勤部政治部主任，冀鲁豫军区政治部组织部部长"):
0.46918297 late: ("解放战争时期，任陕甘宁联防军政治部组织部副部长，联防军后勤部政治部主任，冀鲁豫军区政治部组织部部长"):
===
0.8535181 trational: ("他因战伤截肢，人称“独脚将军。”获一级八一勋章、二级独立自由勋章、一级解放勋章和一级红星功勋荣誉章。"):
0.4918398 late: ("他因战伤截肢，人称“独脚将军。”获一级八一勋章、二级独立自由勋章、一级解放勋章和一级红星功勋荣誉章。"):
===
